# 预训练语义分割模型预测-视频

同济子豪兄：https://space.bilibili.com/1900783

2022-10-18

## 进入 mmsegmentation 主目录

In [1]:
import os
os.chdir('mmsegmentation')

## 视频预测-命令行

In [2]:
!python demo/video_demo.py \
        data/traffic.mp4 \
        configs/segformer/segformer_mit-b5_8x1_1024x1024_160k_cityscapes.py \
        https://download.openmmlab.com/mmsegmentation/v0.5/segformer/segformer_mit-b5_8x1_1024x1024_160k_cityscapes/segformer_mit-b5_8x1_1024x1024_160k_cityscapes_20211206_072934-87a052ec.pth \
        --device cuda:0 \
        --palette cityscapes \
        --output-file outputs/output_pred_traffic.mp4 \
        --opacity 0.5

/home/featurize/work/MMSegmentation教程/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:236: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  'Default ``avg_non_ignore`` is False, if you would like to '
load checkpoint from http path: https://download.openmmlab.com/mmsegmentation/v0.5/segformer/segformer_mit-b5_8x1_1024x1024_160k_cityscapes/segformer_mit-b5_8x1_1024x1024_160k_cityscapes_20211206_072934-87a052ec.pth
OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 7 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
/home/featurize/work/MMSegmentation教程/mmsegmentation/mmseg/models/segmentors/base.py:289: UserWarning: show==False and out_file is not specified, only result image will be returned
  warnings.warn('show==False and out_file is not speci

## 视频预测-Python API

### 导入工具包

In [2]:
import numpy as np
import mmcv
import time
import shutil
from mmseg.apis import inference_segmentor

### 载入模型

In [3]:
# 模型 config 配置文件
config_file = 'configs/segformer/segformer_mit-b5_8x1_1024x1024_160k_cityscapes.py'
# 模型 checkpoint 权重文件
checkpoint_file = 'https://download.openmmlab.com/mmsegmentation/v0.5/segformer/segformer_mit-b5_8x1_1024x1024_160k_cityscapes/segformer_mit-b5_8x1_1024x1024_160k_cityscapes_20211206_072934-87a052ec.pth'

from mmseg.apis import init_segmentor
model = init_segmentor(config_file, checkpoint_file, device='cuda:0')

/home/featurize/work/MMSegmentation教程/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:236: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  'Default ``avg_non_ignore`` is False, if you would like to '


load checkpoint from http path: https://download.openmmlab.com/mmsegmentation/v0.5/segformer/segformer_mit-b5_8x1_1024x1024_160k_cityscapes/segformer_mit-b5_8x1_1024x1024_160k_cityscapes_20211206_072934-87a052ec.pth


### 输入视频路径

In [4]:
input_video = 'data/traffic.mp4'

input_video = 'data/street_20220330_174028.mp4'

### 创建临时文件夹，存放每帧结果

In [5]:
temp_out_dir = time.strftime('%Y%m%d%H%M%S')
os.mkdir(temp_out_dir)
print('创建临时文件夹 {} 用于存放每帧预测结果'.format(temp_out_dir))

创建临时文件夹 20221021231102 用于存放每帧预测结果


### 视频逐帧预测

In [6]:
# 读入待预测视频
imgs = mmcv.VideoReader(input_video)

prog_bar = mmcv.ProgressBar(len(imgs))

# 对视频逐帧处理
for frame_id, img in enumerate(imgs):
    
    ## 处理单帧画面
    result = inference_segmentor(model, img) # 语义分割预测
    temp_path = f'{temp_out_dir}/{frame_id:06d}.jpg' # 保存语义分割预测结果图像至临时文件夹
    model.show_result(img, result, out_file=temp_path, opacity=0.5) # opacity控制透明度，越小，越接近原图

    prog_bar.update() # 更新进度条

# 把每一帧串成视频文件
mmcv.frames2video(temp_out_dir, 'outputs/output_pred.mp4', fps=imgs.fps, fourcc='mp4v')

shutil.rmtree(temp_out_dir) # 删除存放每帧画面的临时文件夹
print('删除临时文件夹', temp_out_dir)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 138/138, 22.7 task/s, elapsed: 6s, ETA:     0s[                                                  ] 0/138, elapsed: 0s, ETA:
删除临时文件夹 20221021231102
